In [ ]:
!pip install tensorflow-addons
!pip install pyyaml h5py
!pip install toolz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.9.2


## Setup

In [ ]:
import os
# os.environ["MODIN_CPUS"] = "8"
# from distributed import Client
# client = Client()
import numpy as np
import math
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
# import allel
from scipy.spatial.distance import squareform
%matplotlib inline   
from toolz import interleave
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.9.2


## Hardware Config

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on TPU  grpc://10.83.233.50:8470
N_REPLICAS: 8


## Prepare the data

In [ ]:
# load genotype
genotypes = pd.read_csv('[path]/genotype_full.txt', sep='\t', index_col=0)
genotypes[genotypes == -1] = 0

headers = genotypes.columns[:]

In [ ]:
genotypes.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_02,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_03,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = genotypes
X.shape

(4390, 28220)

In [ ]:
# hyperparameters
feature_size = X.shape[1]
inChannel = 3
learning_rate = 0.01
weight_decay = 0.00001
embed_dim = 64  # Embedding size for each token
num_heads = 8 # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
regularization_coef_l1 = 1e-4
batch_size = 20
dropout_rate = 0.25

## Convert to tensorflow dataset

In [ ]:
@tf.function()
def add_attention_mask(X_sample, y_sample):
  depth = 3
  mask_size = tf.cast(X_sample.shape[0]*0.5, dtype=tf.int64)
  mask_idx = tf.reshape(tf.random.shuffle(tf.range(X_sample.shape[0]))[:mask_size], (-1, 1))
  updates = tf.math.add(tf.ones(shape=(mask_idx.shape[0]), dtype=tf.int64), 1)
  X_masked = tf.tensor_scatter_nd_update(X_sample, mask_idx, updates)
  return tf.one_hot(X_masked, depth), tf.one_hot(y_sample, depth-1)

In [ ]:
def map_values_1(x):
  return 0 if (x == 0 or x == 1) else 1

def map_values_2(x):
  return 0 if (x == 0 or x == 2) else 1

def get_dataset(x, chunk_start, chunk_end, start_offset, end_offset, batch_size, training=True):
  AUTO = tf.data.AUTOTUNE
  new_chunk_end = x.shape[1]

  dataset = tf.data.Dataset.from_tensor_slices((x,
                                                x[:, start_offset:new_chunk_end-end_offset]))
  
  if training:
    dataset = dataset.shuffle(x.shape[0], reshuffle_each_iteration=True)
    dataset = dataset.repeat()
  
  # Add Attention Mask
  dataset = dataset.map(add_attention_mask, num_parallel_calls=AUTO, deterministic=False)

  # Prefetech to not map the whole dataset
  dataset = dataset.prefetch(AUTO)

  dataset = dataset.batch(batch_size, drop_remainder=True, num_parallel_calls=AUTO)

  return dataset

## Hyper-params


In [ ]:
kr = 1e-5
drop_prec = 0.25

## Model

In [ ]:
def getHLAModel(filter_sizes1=1024, filter_sizes2=512, kernel_size=512):
  inputt = layers.Input(shape=(feature_size, inChannel))
  # x = NewGenoEmbeddings(filter_sizes[0])(inputt)
  x = layers.Conv1D(filter_sizes1, kernel_size=kernel_size, padding="SAME")(inputt)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)
  x = layers.MaxPool1D(2)(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Conv1D(filter_sizes2, kernel_size=kernel_size, padding="SAME")(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)
  x = layers.MaxPool1D(2)(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(8, activation=tf.nn.softmax)(x)
  x = layers.Reshape((feature_size, inChannel-1))(x)
  model = tf.keras.models.Model(inputt, x)
  return model

In [ ]:
model = getHLAModel()
model.build((1, feature_size, inChannel))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 conv1d (Conv1D)             (None, 28220, 1024)       1573888   
                                                                 
 batch_normalization (BatchN  (None, 28220, 1024)      4096      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 28220, 1024)       0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14110, 1024)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14110, 1024)       0     

In [ ]:
def create_model():
  model =  getHLAModel()
  optimizer = tfa.optimizers.LAMB(learning_rate=learning_rate)
  model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=tf.keras.metrics.CategoricalAccuracy())
  return model

In [ ]:
METRIC = "val_categorical_accuracy"

def create_callbacks(kfold=0, metric = METRIC):

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='auto',
        factor=0.5,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='auto',
        patience= 10, 
        verbose=1,
        restore_best_weights=True
    )
    
    callbacks = [
                 reducelr,
                 earlystop
                 ]         
    
    return callbacks

## Training

In [ ]:
# A TPU V3-8 has 8 computing cores, the global batch size will be 1/16 x 8 = 8/128
BATCH_SIZE_BASE = 6
# Training configuration
BATCH_SIZE = BATCH_SIZE_BASE * N_REPLICAS if TPU else 32
BATCH_SIZE

48

In [ ]:
# With constraint
missing_perc = 0.1
# N_SPLITS = 5
# batch_size = 50

N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold < 3:
    continue
  accuracies = []
  print(f"Training using fold {fold}")
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=0)
  for missing_perc in [
                         0.01, 0.05,
                         0.1,
                         0.2
                         ]:
    save_name = f"[path]/YEAST/HLA_DEEP/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0])), position=0, leave=True):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # predict
      predict_onehot = model.predict(test_X_missing[i:i+1, :, :], verbose=0)
      # only care the missing position
      predict_missing_onehot = predict_onehot[0:1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

Training using fold 3
Restoring model weights from the end of the best epoch: 28.
Epoch 38: early stopping


  0%|          | 0/1463 [00:00<?, ?it/s]

1/1 [==============================] - 24s 24s/step


  0%|          | 1/1463 [01:30<36:33:31, 90.02s/it]

1/1 [==============================] - 1s 1s/step


...

1/1 [==============================] - 1s 1s/step


100%|█████████▉| 1462/1463 [34:11<00:01,  1.38s/it]

1/1 [==============================] - 1s 1s/step


100%|██████████| 1463/1463 [34:13<00:00,  1.40s/it]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9980: 
Sensitivity: 0.9979876108422244
Specificity: 0.9979876108422244
F1-score macro: 0.9979883932376388
F1-score micro: 0.9979884154042086


In [ ]:
# With constraint
missing_perc = 0.1
# N_SPLITS = 5
# batch_size = 50

N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold < 2:
    continue
  accuracies = []
  print(f"Training using fold {fold}")
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=0)
  for missing_perc in [
                         0.01, 0.05,
                         0.1,
                         0.2
                         ]:
    save_name = f"[path]/YEAST/HLA_DEEP/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0])), position=0, leave=True):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
      predict_onehot = model.predict(test_X_missing[i:i+1, :, :])
      # only care the missing position
      predict_missing_onehot = predict_onehot[0:1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

Training using fold 2
Restoring model weights from the end of the best epoch: 30.
Epoch 40: early stopping


  0%|          | 0/1463 [00:00<?, ?it/s]

1/1 [==============================] - 24s 24s/step


  0%|          | 1/1463 [01:31<37:12:25, 91.62s/it]

1/1 [==============================] - 1s 1s/step


...

100%|█████████▉| 1462/1463 [33:13<00:01,  1.33s/it]

1/1 [==============================] - 1s 1s/step


100%|██████████| 1463/1463 [33:14<00:00,  1.36s/it]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9980: 
Sensitivity: 0.9979644162550645
Specificity: 0.9979644162550645
F1-score macro: 0.997964794368736
F1-score micro: 0.9979647995706036
Training using fold 3


KeyboardInterrupt: ignored

In [ ]:
# With constraint
missing_perc = 0.1

N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  accuracies = []
  print(f"Training using fold {fold}")
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=0)
  for missing_perc in [
                         0.01, 0.05,
                         0.1,
                         0.2
                         ]:
    save_name = f"[path]/YEAST/HLA_DEEP/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # predict
      predict_onehot = model.predict(test_X_missing[i:i+1, :, :])
      # only care the missing position
      predict_missing_onehot = predict_onehot[0:1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

Training using fold 1
Restoring model weights from the end of the best epoch: 23.
Epoch 33: early stopping


  0%|          | 0/1464 [00:00<?, ?it/s]

1/1 [==============================] - 23s 23s/step


  0%|          | 1/1464 [01:35<38:49:44, 95.55s/it]

1/1 [==============================] - 1s 1s/step


...

100%|█████████▉| 1463/1464 [34:21<00:01,  1.40s/it]

1/1 [==============================] - 1s 1s/step


100%|██████████| 1464/1464 [34:23<00:00,  1.41s/it]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9980: 
Sensitivity: 0.9979540310024224
Specificity: 0.9979540310024224
F1-score macro: 0.9979539642445858
F1-score micro: 0.9979539662991407
Training using fold 2


ResourceExhaustedError: ignored